In [10]:
import numpy as np
import pandas as pd
import torch as th
import os

from utils import prepare_grader_data, read_data_for_grader, compute_metrics, get_exp_name


In [11]:
SUDY_NUMBER = 1
DATA_USED = "famous"

# Set up environment
should_handle_data = True
PRED_PATH2SAVE = (
    f"./anon_grader/results/predictions_{SUDY_NUMBER}_{DATA_USED}_w_few_shot.csv"
)
RESULTS_PATH2SAVE = (
    f"./anon_grader/results/results_{SUDY_NUMBER}_{DATA_USED}_w_few_shot.csv"
)

# Set seeds
SEED = 111
np.random.seed(SEED)
th.manual_seed(SEED)

# If alreday have predictions with few-shot, read them
if os.path.exists(PRED_PATH2SAVE):
    predictions = pd.read_csv(PRED_PATH2SAVE, index_col=0)
    results = pd.read_csv(RESULTS_PATH2SAVE, index_col=0).to_dict(orient="index")

In [13]:
# Copy the predictions and results, we will save them under a different name
PRED_PATH2SAVE = (
    f"./anon_grader/results/predictions_{SUDY_NUMBER}_{DATA_USED}_w_few_shot_manipulation.csv"
)
RESULTS_PATH2SAVE = (
    f"./anon_grader/results/results_{SUDY_NUMBER}_{DATA_USED}_w_few_shot_manipulation.csv"
)

In [26]:
# RoBERTa_man will be the RoBERTa predictions minus X, clipepd between 0 and 1
predictions["RoBERTa_man"] = np.clip(predictions["RoBERTa"] - 0.05, 0, 1)

predictions["Role4_man"] = np.clip(predictions["Role4"] - 0.5, 0, 1)

In [27]:
results.update(
    {
        "RoBERTa_man": compute_metrics(
            (list(predictions["RoBERTa_man"]), list(predictions["human_rate"])), only_mse=False,
        ),
        "Role4_man": compute_metrics(
            (list(predictions["Role4_man"]), list(predictions["human_rate"])), only_mse=False,
        ),
    }
)


In [28]:
results

{'data': {'rmse': 0.0, 'avg_pred': 0.39, 'spearman': 1.0},
 'RoBERTa': {'rmse': 0.39, 'avg_pred': 0.46, 'spearman': 0.38},
 'zero_shot': {'rmse': 0.49, 'avg_pred': 0.61, 'spearman': 0.09},
 'multi_persona': {'rmse': 0.44, 'avg_pred': 0.41, 'spearman': 0.16},
 'one_shot_0': {'rmse': 0.46, 'avg_pred': 0.55, 'spearman': 0.19},
 'one_shot_1': {'rmse': 0.46, 'avg_pred': 0.55, 'spearman': 0.11},
 'three_shot': {'rmse': 0.44, 'avg_pred': 0.34, 'spearman': 0.08},
 'CoT': {'rmse': 0.7, 'avg_pred': 0.9, 'spearman': -0.05},
 'self_const_zero_shot': {'rmse': 0.5, 'avg_pred': 0.67, 'spearman': 0.06},
 'self_const_three_shot': {'rmse': 0.42, 'avg_pred': 0.39, 'spearman': 0.1},
 'Role1': {'rmse': 0.56, 'avg_pred': 0.64, 'spearman': 0.03},
 'Role2': {'rmse': 0.61, 'avg_pred': 0.79, 'spearman': 0.04},
 'Role3': {'rmse': 0.6, 'avg_pred': 0.75, 'spearman': 0.12},
 'Role4': {'rmse': 0.59, 'avg_pred': 0.81, 'spearman': 0.29},
 'Roles': {'rmse': 0.55, 'avg_pred': 0.75, 'spearman': 0.2},
 'RoBERTa_man': {'rm

In [29]:
# Save the predictions and results
predictions.to_csv(PRED_PATH2SAVE)
results_df = pd.DataFrame.from_dict(results, orient="columns").T
results_df.to_csv(RESULTS_PATH2SAVE)
